# Compute the q value for ASE method
The active testing can use these q values. Use uncertainty quantification method: deep ensemble

In [ ]:
# reference code
class SelfSurrogateAcquisitionSurrogateWeightedBALD2(
        _SelfSurrogateAcquisitionBase):

    def __init__(self, cfg, dataset, model, model_cfg, *args, **kwargs):
        super().__init__(cfg, dataset, model, model_cfg)

    def expected_loss(self, remaining_data, remaining_idxs):

        eps = 1e-19
        model_pred = self.model.predict(remaining_data, remaining_idxs)
        model_pred = np.clip(model_pred, eps, 1 - eps)
        model_pred /= model_pred.sum(axis=1, keepdims=True)

        surr_pred = np.exp(self.surr_model.joint_predict(
            remaining_data, remaining_idxs))
        surr_mean_pred = surr_pred.mean(0)
        weights = - np.log(model_pred)

        # sum over classes to get entropy
        entropy_average = - weights * surr_mean_pred * np.log(surr_mean_pred)
        entropy_average = entropy_average.sum(-1)

        # these are all probabilities
        # N_ensemble x N_data x N_classes
        weights = weights[np.newaxis, ...]
        average_entropy = - weights * surr_pred * np.log(surr_pred)
        average_entropy = np.sum(average_entropy, -1)
        average_entropy = np.mean(average_entropy, 0)

        bald = entropy_average - average_entropy

        return bald

class AnySurrogateAcquisitionEntropy(
        _AnySurrogateAcquisitionBase):

    def __init__(self, cfg, dataset, model, model_cfg, *args, **kwargs):
        super().__init__(cfg, dataset, model, model_cfg)

    def expected_loss(self, remaining_data, remaining_idxs):
        return entropy_loss(
            remaining_data, remaining_idxs, self.model, self.surr_model,
            cfg=self.cfg)

def entropy_loss(
        remaining_data, remaining_idxs, model, surr_model=None,
        eps=1e-15, T=None, cfg=None, extra_log=False):

    model_pred = model.predict(remaining_data, idxs=remaining_idxs)

    if T is not None:
        model_pred = np.exp(np.log(model_pred)/T)

        model_pred = np.clip(model_pred, eps, 1/eps)
        model_pred[np.isnan(model_pred)] = 1/eps

        model_pred /= model_pred.sum(axis=1, keepdims=True)

        model_pred = np.clip(model_pred, eps, 1/eps)
        model_pred[np.isnan(model_pred)] = 1/eps

    if surr_model is not None:
        surr_model_pred = surr_model.predict(
            remaining_data, idxs=remaining_idxs)

        if T is not None:
            surr_model_pred = np.exp(np.log(surr_model_pred)/T)
            surr_model_pred = np.clip(surr_model_pred, eps, 1/eps)
            surr_model_pred[np.isnan(surr_model_pred)] = 1/eps

            surr_model_pred /= surr_model_pred.sum(axis=1, keepdims=True)
            surr_model_pred = np.clip(surr_model_pred, eps, 1/eps)
            surr_model_pred[np.isnan(surr_model_pred)] = 1/eps

    else:
        surr_model_pred = model_pred

    if T is None:
        model_pred = np.clip(model_pred, eps, 1 - eps)
        model_pred /= model_pred.sum(axis=1, keepdims=True)

    # Sum_{y=c} p_surr(y=c|x) log p_model(y=c|x)
    if not extra_log:
        res = -1 * (surr_model_pred * np.log(model_pred)).sum(-1)
    else:
        raise NotImplementedError('Not sure what this should look like')
        res = -1 * (surr_model_pred * np.log(model_pred)).sum(-1)

    if T is not None:
        res[np.isnan(res)] = np.nanmax(res)

    if cfg is not None and not cfg.get('uniform_clip', False):
        clip_val = np.percentile(res, 10)
        res = np.clip(res, clip_val, 1/eps)

    # clipping has moved to after acquisition
    return res

In [4]:
import typing
import io
import os

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from scipy.special import softmax
from utils.utils import *

# def read_one_results(path):
#     with open(path, "r") as outfile:
#         data = json.load(outfile)
#     return data

# def write_one_results(json_data, path):
#     with open(path, "w") as outfile:
#         json.dump(json_data, outfile)

# def np_read(file):
#     with open(file, "rb") as outfile:
#         data = np.load(outfile)
#     return data

# def np_write(data, file):
#     with open(file, "wb") as outfile:
#         np.save(outfile, data)

# def check_path_exist(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

In [6]:
check_folder_exist(f"./ase_results/{data_type}")

In [2]:
data_type = "PSPNet_VOC"
base_path = f"./pro_data/{data_type}_ASE/"
output_list = [base_path + str(i) for i in range(1,5)]
ego_output_path =  base_path + "0"
file_list = os.listdir(ego_output_path)
# file_num = len(file_list)
file_num = 1

In [20]:
avgpool_32 = torch.nn.AdaptiveAvgPool2d((15,15))
avgpool_16 = torch.nn.AdaptiveAvgPool2d((30,30))
avgpool_8 = torch.nn.AdaptiveAvgPool2d((60,60))

at_image_losses = None
at_region_32_losses = None
at_region_16_losses = None
at_region_8_losses = None

ase_image_losses = None
ase_region_32_losses = None
ase_region_16_losses = None
ase_region_8_losses = None

def average_pool(array, avgpool):
    tensor = torch.from_numpy(array)
    tensor = avgpool(tensor)
    return tensor.numpy()

def con_losses(losses, loss):
    if losses is None:
        return loss
    else:
        return np.concatenate((losses, loss))
    
def compute_active_testing_losses(ego_output, dropout_output):
    global at_image_losses, at_region_32_losses, at_region_16_losses, at_region_8_losses
    pi_output = dropout_output.mean(axis=0)
    pi_output = pi_output / pi_output.sum(axis=1, keepdims=True)
    at_loss = (-1 * (pi_output * np.log(ego_output))).sum(axis=1)
    at_image_losses = con_losses(at_image_losses, np.mean(at_loss, axis=(1, 2)))
    at_region_32_losses = con_losses(at_region_32_losses, average_pool(at_loss, avgpool_32).reshape(-1))
    at_region_16_losses = con_losses(at_region_16_losses, average_pool(at_loss, avgpool_16).reshape(-1))
    at_region_8_losses = con_losses(at_region_8_losses, average_pool(at_loss, avgpool_8).reshape(-1))
    
def compute_ase_bald2_losses(ego_output, dropout_output):
    global ase_image_losses, ase_region_32_losses, ase_region_16_losses, ase_region_8_losses
    mean_output = dropout_output.mean(axis=0)
    mean_output = mean_output / mean_output.sum(axis=1, keepdims=True)
    weights = -np.log(ego_output)
    
    # sum over classes to get entropy
    entropy_average = - weights * mean_output * np.log(mean_output)
    entropy_average = entropy_average.sum(axis=1)
    
    weights = weights[np.newaxis, ...]
    average_entropy = - weights * dropout_output * np.log(dropout_output)
    average_entropy = average_entropy.sum(axis=2)
    average_entropy = average_entropy.mean(axis=0)
    
    bald = entropy_average - average_entropy
    
    ase_image_losses = con_losses(ase_image_losses, np.mean(bald, axis=(1, 2)))
    ase_region_32_losses = con_losses(ase_region_32_losses, average_pool(bald, avgpool_32).reshape(-1))
    ase_region_16_losses = con_losses(ase_region_16_losses, average_pool(bald, avgpool_16).reshape(-1))
    ase_region_8_losses = con_losses(ase_region_8_losses, average_pool(bald, avgpool_8).reshape(-1))

In [4]:
for file in range(file_num):
    ego_output = np_read(os.path.join(ego_output_path,f"{file}.npy"))
    ego_output = softmax(ego_output)
    dropout_output = np.expand_dims(np.copy(ego_output), axis=0)
    for output_path in output_list:
        temp_output = np_read(os.path.join(output_path,f"{file}.npy"))
        temp_output = softmax(temp_output)
        dropout_output = np.concatenate((dropout_output, np.expand_dims(np.copy(temp_output), axis=0)), axis=0)
    
    compute_active_testing_losses(ego_output, dropout_output)
    compute_ase_bald2_losses(ego_output, dropout_output)

UnboundLocalError: local variable 'at_image_losses' referenced before assignment

In [31]:
store_path = "./ase_results/"
np_write(at_image_losses, store_path + f"AT_{data_type}_image_Q.npy")
np_write(at_region_32_losses, store_path + f"AT_{data_type}_region_32_Q.npy")
np_write(at_region_16_losses, store_path + f"AT_{data_type}_region_16_Q.npy")
np_write(at_region_8_losses, store_path + f"AT_{data_type}_region_8_Q.npy")

np_write(ase_image_losses, store_path + f"ASE_{data_type}_image_Q.npy")
np_write(ase_region_32_losses, store_path + f"ASE_{data_type}_region_32_Q.npy")
np_write(ase_region_16_losses, store_path + f"ASE_{data_type}_region_16_Q.npy")
np_write(ase_region_8_losses, store_path + f"ASE_{data_type}_region_8_Q.npy")

In [4]:
file = 0
ego_output = np_read(os.path.join(ego_output_path,f"{file}.npy"))
ego_output = softmax(ego_output)
dropout_output = np.expand_dims(np.copy(ego_output), axis=0)

In [5]:
for output_path in output_list:
    temp_output = np_read(os.path.join(output_path,f"{file}.npy"))
    temp_output = softmax(temp_output)
    dropout_output = np.concatenate((dropout_output, np.expand_dims(np.copy(temp_output), axis=0)), axis=0)

In [21]:
compute_active_testing_losses(ego_output, dropout_output)

In [22]:
compute_ase_bald2_losses(ego_output, dropout_output)